In [88]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits import mplot3d
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os

In [89]:
df_train = pd.read_excel("/home/rafael/Documentos/FACOM/Douturado/Doutorado/webcrawler/medicamentos.xlsx",index_col=0,dtype=str)
df_train.head()
list_preprossing=[]

In [90]:
for dta in range(df_train.shape[0]):
    # value=str(df_train.iloc[dta,0])
    value=str(df_train.iloc[dta,0])+" "+str(df_train.iloc[dta,1]) +" "+str(df_train.iloc[dta,2])+" "+str(df_train.iloc[dta,3])
    list_preprossing.append(value)
list_preprossing=np.array(list_preprossing)
print(list_preprossing)

In [91]:
# print(list_preprossing)
print(list_preprossing.shape)

In [92]:
embedder = SentenceTransformer('all-mpnet-base-v2',device='cuda')

In [93]:
corpus_embeddings = embedder.encode(list_preprossing)

In [94]:
for data in corpus_embeddings:
    print(data.shape)

In [95]:
tsne = TSNE(random_state = 42, n_components=2)
pca = PCA(n_components=2)
lda = LinearDiscriminantAnalysis(n_components=2)

In [96]:
embeddings2d_pca = pca.fit_transform(corpus_embeddings)
# embeddings2d_lda = lda.fit_transform(corpus_embeddings,list_preprossing)


In [97]:
knn_4 = KMeans(n_clusters=4,n_init=40)
modelo_knn_3 = knn_4.fit_predict(embeddings2d_pca)

In [98]:
list_name =list(df_train.iloc[:,0])#lista com os nomes do medicamento
# print(list_name)
# print(modelo_knn_3)
classes=[] #dicionario para armazenar a classe. key(medicamento):value(classe)
color = {0:'.b',1:'.r',2:'.c',3:'.y',4:'xb',5:'xr',6:'xc',7:'xy',8:'xk',9:'xb',10:'xr',11:'xc',12:'xy',13:'xk'} #dicionario  de classe e cores
colors=[ color[x] for x in modelo_knn_3] # modelo_knn_3 retorna uma lista de valores [2,1,3,4]
fig,ax = plt.subplots(1,figsize=(10,10))
for i, cl_xy in enumerate(zip(colors,embeddings2d_pca)): #embeddings2d possui x e y
    cl=cl_xy[0]
    xy=cl_xy[1]
    ax.plot(xy[0],xy[1],cl)
    # print(df_train.iloc[i].name)
    classes.append({'nome':df_train.iloc[i].name,'ativo':list_name[i],"classes":df_train['classes'].iloc[i],"classe_pred":modelo_knn_3[i]})
    # classes.append({'nome':df_train.iloc[i].name,'ativo':list_name[i],"classes":df_train['classes'].iloc[i],"classe_pred":modelo_knn_3[i]})

# for i, cl_xy in enumerate(zip(colors,embeddings2d)): #embeddings2d possui x e y
#     cl=cl_xy[0]
#     xy=cl_xy[1]
#     ax.plot(xy[0],xy[1],cl)
#     print(df_train.iloc[i].name)
#     classes.append({'nome':df_train.iloc[i].name,'ativo':list_name[i],"classes":df_train['classes'].iloc[i],"classe_pred":modelo_knn_3[i]})
fig.savefig('my_plot.png')


In [99]:
print(classes)
# print(df_train)
# frame_=pd.DataFrame({"x":embeddings2d[:,0]})
# np.savetxt("embedding2d.csv",np.concatenate([[[1,2,3],[1,2,3],[1,2,3]],[0,0,0]],axis=0),delimiter=",")


In [100]:
frame_result = pd.DataFrame(classes).sort_values("classe_pred",axis=0)

In [101]:
frame_result

In [102]:
frame_result.to_csv("Save_EuroFarma.csv")